In [88]:
from pymongo import MongoClient
import pandas as pd
import json
from pandas.io.json import json_normalize
from googlemaps import convert
import googlemaps
from datetime import datetime
import requests
from dotenv import load_dotenv
import os
from keplergl import KeplerGl
import operator
from functools import reduce

In [2]:
client= MongoClient ("localhost:27017")

In [3]:
db = client.get_database("Ironhack")

In [4]:
collection = db.get_collection("companies")

In [5]:
load_dotenv()

True

In [6]:
tokg = os.getenv("tokg")


In [7]:
gmaps = googlemaps.Client(key= tokg )

# GeoSpatial Data Project:

1. Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
2. 30% of the company staff have at least 1 child.
3. Developers like to be near successful tech startups that have raised at least 1 Million dollars.
4. Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
5. Account managers need to travel a lot.
6. Everyone in the company is between 25 and 40, give them some place to go party.
7. The CEO is vegan.
8. If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
9. The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.


Para ello utilizaremos dos herramientas principales, por un lado Mongodb y la base de datos "companies" y por otro la api de Google Places, finalmente mostraremos los resultados obtenidos utilizando la libreria Kepler.

## Empezamos...


### Filtrado inicial y localización:

#### Lo primero que vamos a hacer es empezar a filtrar las companies por dos de sus keys: category code, que nos dice a qué se dedica, filtraremos por "design", como quieren los trabajadores; y total money raised, con un regex para que incluya "M" de millones


In [8]:
#creamos 2 condicionantes y luego los enlistamos

cond= {"total_money_raised" : {"$regex" : "M"}}
# cond0= {""}, la moneda solo está en IPO, y no todas tienen ipo...
cond1= {"category_code": "design"}

In [9]:
list ((collection.find (({"$and" : [cond, cond1]}) , {"name":1, "tag_list": 1 , "_id":0})).limit (50))

[{'name': '99designs',
  'tag_list': 'graphic-design, crowdsourcing, online-marketplace, marketplace, crowdsourced-design'},
 {'name': 'Kickstarter',
  'tag_list': 'crowdfunding, funding, entrepreneurship, crowdsourcing'},
 {'name': 'Moonfruit', 'tag_list': ''}]

In [10]:
#esto solo era una comprobación
list ((collection.find (({"$and" : [cond1]}) , {"name":1,"total_money_raised": 1, "tag_list": 1 , "_id":0})).limit (50))

[{'name': '99designs',
  'tag_list': 'graphic-design, crowdsourcing, online-marketplace, marketplace, crowdsourced-design',
  'total_money_raised': '$35M'},
 {'name': 'Graticle',
  'tag_list': 'web-design, web-hosting, graphic-design, web-development, seo, search-engine-optimization',
  'total_money_raised': '$0'},
 {'name': 'Kickstarter',
  'tag_list': 'crowdfunding, funding, entrepreneurship, crowdsourcing',
  'total_money_raised': '$10M'},
 {'name': 'Moonfruit', 'tag_list': '', 'total_money_raised': '$2.25M'}]

#### Vemos que solo hay 3 empresas que cumplan con ser empresas de diseño y haber recaudado 1 millón, me centraré en ellas, primero buscaré su ubicación


In [11]:
# busco en el offices para  ver la dirección
list ((collection.find (({"$and" : [cond, cond1]}) , { "name": 1, "offices": 1 , "_id":0})))

[{'name': '99designs',
  'offices': [{'description': 'United States (HQ)',
    'address1': '447 Battery St.',
    'address2': '3rd Floor',
    'zip_code': '94111',
    'city': 'San Francisco',
    'state_code': 'CA',
    'country_code': 'USA',
    'latitude': 37.7955307,
    'longitude': -122.4005983},
   {'description': 'Australia',
    'address1': '204 Wellington St.',
    'address2': '',
    'zip_code': '3065',
    'city': 'Collingwood',
    'state_code': None,
    'country_code': 'AUS',
    'latitude': -37.8026587,
    'longitude': 144.9868546},
   {'description': 'Europe',
    'address1': 'Schlesische Str. 29-30',
    'address2': '',
    'zip_code': '',
    'city': 'Berlin',
    'state_code': None,
    'country_code': 'DEU',
    'latitude': 52.4986199,
    'longitude': 13.4469031}]},
 {'name': 'Kickstarter',
  'offices': [{'description': '',
    'address1': '58 Kent St',
    'address2': '',
    'zip_code': '11222',
    'city': 'Brooklyn',
    'state_code': 'NY',
    'country_code'

In [12]:
# le añado una condición, porque si no lo añado como condicionante, no coge la salida .$ :1 que pongo en el siguiente paso.
# creo que lo que voy a hacer hasta llegar al df con las ciudades que quiero es como matar moscas a cañonazos, pero no consigo 
# llegar a la ciudad de otra forma, uno de los resultados tiene 3 ciudades y no llego al que me interesa.
cond2= {"offices": {"$exists": True }}

In [13]:
lista1=list ((collection.find (({"$and" : [cond, cond1, cond2]}) , {"name":1, "offices.$": 1, "_id":0})))

In [14]:
# Por favor, en la correción, si podéis explicarme cómo se usaría el json normalize lo agradecería.
data= pd.DataFrame (lista1)
data

,name,offices
0,99designs,"[{'description': 'United States (HQ)', 'addres..."
1,Kickstarter,"[{'description': '', 'address1': '58 Kent St',..."
2,Moonfruit,"[{'description': 'UK Office', 'address1': '', ..."


In [15]:
df1=pd.DataFrame (data.offices[0])
df2=pd.DataFrame (data.offices[1])
df3=pd.DataFrame (data.offices[2])

In [16]:
df3

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,UK Office,,,,London,None,GBR,None,None


In [17]:
df= pd.concat ([df1,df2,df3], ignore_index=True)
df

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,United States (HQ),447 Battery St.,3rd Floor,94111,San Francisco,CA,USA,37.795531,-122.400598
1,,58 Kent St,,11222,Brooklyn,NY,USA,NaN,NaN
2,UK Office,,,,London,None,GBR,NaN,NaN


#### Ya tenemos los datos necesarios, pero para cada una de las empresas tenemos tipos de datos distintos, inicialmente cogí las ciudades, pero creo que será mas certero buscar la direccion de moonfruit, si busco london en geocode me dará una ubicación perfecta para cualquier tipo de búsqueda

In [31]:
#copiando de las clases, pero cambiando latt por longt.
def geocode(address):
    """
   Saca las coordenadas de una dirección que le des.
    """
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    try:
        return {
           "type":"Point",
            "coordinates":[float(data["latt"]),float(data["longt"])]}
    except:
        return data


In [32]:
bkyn= geocode ("58 Kent St, brooklyn")
sfan= geocode ("447 Battery St., San Francisco, US United States of America")
#london= geocode ( "london"), inicialmente lo tenía así, pero creo que hay que afinar ésta unicación
bkyn

{'type': 'Point', 'coordinates': [40.73132, -73.95009]}

#### Le vamos a preguntar a google places por Moonfruit, nos arroja 3 resultados, sus 3 sucursales. Teniendo que la api de google places siempre ordena por relevancia a no ser que le digamos lo contrario, nos quedamos con el primer resultado

In [33]:
london1=gmaps.places(
    
        query= "moonfruit"
)


In [34]:
london= london1["results"][0]["geometry"]["location"]
london

{'lat': 51.5214517, 'lng': -0.1420567}

### Geoquerys

#### Una vez que tengo las localizaciones, vamos a preguntarle a google por el resto de necesidades de la plantilla, después las meteremos en un dataframe y las puntuaremos de alguna manera aún por determinar.

#### Empezamos con el número de starbucks, le vamos a preguntar a google places con "nearby", que nos devuelve los lugares que cumplan los parámetros :

In [35]:
starsfan=gmaps.places_nearby(
    
    location = sfan["coordinates"],
    radius = 200,
    name = "Starbucks"
    
)

In [36]:
starsbkyn=gmaps.places_nearby(
    
    location = bkyn["coordinates"],
    radius = 200,
    name = "Starbucks"
    
)

In [37]:
starslond= gmaps.places_nearby(
    
        location= london ,
        name ='starbucks',
        radius=200, 
)

#he probado indistintamente "name" y "keyword" y con mayúsculas y minúsculas con los mismos resultados

In [38]:
len (starslond["results"])

5

#### Las consultas a Google Places nos devulven siempre un diccionario de 3 valores, donde el que nos interesa es results, que es donde nos dá los establecimientos. El len nos dá la densidad de establecimientos en el radio seleccionado, será el valor que usaré para ponderar y dar con la ciudad indicada. 

#### Continuamos con las necesidades, en este caso nos centraremos en los colegios cercanos

In [66]:
neneslond = gmaps.places_nearby(
        location= london ,
        name =["kindergarten","school" , "primary school", "high school"], 
        radius=1000
        )

In [67]:
nenesfan=gmaps.places_nearby(
    
    location = sfan["coordinates"],
    radius = 1500,
    name =["kindergarten","school" , "primary school", "high school"],
    
    )

In [68]:
nenesbkyn=gmaps.places_nearby(
    
    location = bkyn["coordinates"],
    radius = 1500,
    name =["kindergarten","school" , "primary school", "high school"],
    
)

#### Para los colegios, los resultados obtenidos son bastante pobres, así que incluiré otra forma de puntuarlos, vamos a calcular la distancia máxima en la que nuestros empleados tendrán al menos 10 instituciones educativas

In [42]:
neneslond2 = gmaps.places_nearby(
        location= london,
        keyword =["kindergarten","school" , "primary school", "high school", "college"],
        rank_by= "distance"
        )
#esto me da los 20 primeros resultados rankeados por distancia, voy a elegir el décimo, y usar su distancia al punto de origen
# como un elemento más para el cálculo ponderado, los coles mas cercanos eran insuficientes.

In [43]:
def loc_decimo (dicfromgoogle):
    """ Recibe como parámetro una respuesta de googleapi y retorna la localización del décimo elemento"""
    return dicfromgoogle ["results"][9]["geometry"]["location"]

In [44]:
last_nenes_lond= loc_decimo (neneslond2)
last_nenes_lond

{'lat': 51.4740438, 'lng': -0.1573354}

In [45]:
def distancia_entre (origen, destino):
   
    """
    Recibe como parámetros 2 coordenadas y nos devuelve la distancia entre ellas en metros y en valor numérico
    
    :param origins: One or more addresses, Place IDs, and/or latitude/longitude
        values, from which to calculate distance and time. Each Place ID string
        must be prepended with 'place_id:'. If you pass an address as a string,
        the service will geocode the string and convert it to a
        latitude/longitude coordinate to calculate directions.
    
    :type origins: a single location, or a list of locations, where a
        location is a string, dict, list, or tuple
    
    :param destinations: One or more addresses, Place IDs, and/or lat/lng values
        , to which to calculate distance and time. Each Place ID string must be
        prepended with 'place_id:'. If you pass an address as a string, the
        service will geocode the string and convert it to a latitude/longitude
        coordinate to calculate directions.
    
    :type destinations: a single location, or a list of locations, where a
        location is a string, dict, list, or tuple
    """ #copiado de la función original
    averquenossale=gmaps.distance_matrix (origen, destino) #esta sería la función original de la librería
    return averquenossale["rows"][0]["elements"][0]["distance"]["value"]# y le decimos que nos retorne lo que nos interesa

In [46]:
radio_nenes_lond = distancia_entre (london , last_nenes_lond)
radio_nenes_lond # es decir, los 10 colegios más cercanos se encuentran en un radio de 5800 metros

7850

In [47]:
nenesbkyn2 = gmaps.places_nearby(
        location= bkyn["coordinates"] ,
        keyword=["kindergarten","school" , "primary school", "high school"],
        rank_by= "distance"
        )

In [48]:
last_nenes_bkyn= loc_decimo (nenesbkyn2)
last_nenes_bkyn

{'lat': 40.7188322, 'lng': -73.9982562}

In [49]:
# cambio el formato que no me lo coge en matrix
stringbkyn= str (bkyn["coordinates"][0])+ "," +str (bkyn["coordinates"][1])
stringsfan= str (sfan["coordinates"][0])+ "," +str (sfan["coordinates"][1])


In [50]:
radio_nenes_bkyn = distancia_entre (stringbkyn, last_nenes_bkyn)
radio_nenes_bkyn

7091

In [51]:
nenessfan2 = gmaps.places_nearby(
        location= sfan["coordinates"] ,
        keyword =["kindergarten","school" , "primary school", "high school"],
        rank_by= "distance"
        )

In [52]:
last_nenes_sfan= loc_decimo (nenessfan2)

In [53]:
stringsfan= str (sfan["coordinates"][0])+ "," +str (sfan["coordinates"][1])


In [54]:
radio_nenes_sfan = distancia_entre (stringsfan, last_nenes_sfan)
radio_nenes_sfan

4080

#### Vamos con los veganos, misma operativa que con los colegios, añadiendo el type

In [55]:
vegansbkyn=gmaps.places_nearby(
    
    location = bkyn["coordinates"],
    radius = 1000,
    keyword=["vegan"],
    type= ["restaurant"]
    
)

In [56]:
veganssfan=gmaps.places_nearby(
    
    location = sfan["coordinates"],
    radius = 1000,
    keyword=["vegan"],
    type= ["restaurant"]
    
)

In [57]:
veganslond = gmaps.places_nearby(
        location= london ,
        radius = 1000,
        keyword=["vegan"],
        type= ["restaurant"]
 )

#### Un poquito de fiesta para cuando acaben las restricciones

In [58]:
partylond = gmaps.places_nearby(
        location= london ,
        radius = 1000,
        keyword=["party"],
        type= ["bar"]
 )

In [59]:
partysfan=gmaps.places_nearby(
    
    location = sfan["coordinates"],
    radius = 1000,
    keyword=["party"],
    type= ["bar"]
    
)

In [60]:
partybkyn=gmaps.places_nearby(
    
    location = bkyn["coordinates"],
    radius = 1000,
    keyword=["party"],
    type= ["bar"]
    
)

# Puntuaciones

### Vamos a meter los resultados en un Dataframe, para posteriormente añadir columnas con la puntuación de la ciudad para cada campo determinado y finalmente realizaremos el cálculo final

In [61]:
dfpuntuacion= pd.DataFrame ()

In [69]:
dfpuntuacion["Ubicacion"]= ["San_Francisco", "Brooklyn", "London"]
dfpuntuacion["Densidad_Starbucks_200mtr"]= [len (starsfan["results"]),len (starsbkyn["results"]),len (starslond["results"])] 
dfpuntuacion["Densidad_Bares_1000mtrs"]= [len (partysfan["results"]),len (partybkyn["results"]),len (partylond["results"])] 
dfpuntuacion["Densidad_Veganos_1000mtrs"]= [len (veganssfan["results"]),len (vegansbkyn["results"]),len (veganslond["results"])] 
dfpuntuacion["Densidad_Colegios_1500mtrs"]= [len (nenesfan["results"]),len (nenesbkyn["results"]),len (neneslond["results"])] 
dfpuntuacion["Radio_para_encontrar_10_coles"]= [radio_nenes_sfan, radio_nenes_bkyn, radio_nenes_lond]

In [70]:
dfpuntuacion

,Ubicacion,Densidad_Starbucks_200mtr,Densidad_Bares_1000mtrs,Densidad_Veganos_1000mtrs,Densidad_Colegios_1500mtrs,Radio_para_encontrar_10_coles,densidad_de_cosas
0,San_Francisco,9,20,6,3,4080,44
1,Brooklyn,1,10,1,1,7091,13
2,London,5,18,20,1,7850,43


In [71]:
dfpuntuacion ["densidad_de_cosas"]= dfpuntuacion["Densidad_Starbucks_200mtr"] + dfpuntuacion["Densidad_Bares_1000mtrs"] + dfpuntuacion["Densidad_Veganos_1000mtrs"] + dfpuntuacion["Densidad_Colegios_1500mtrs"] 

In [72]:
dfpuntuacion.densidad_de_cosas # además que aquí lo que importa es que el jefe coma bien

0    38
1    13
2    44
Name: densidad_de_cosas, dtype: int64

### Y el ganador es: Moonfruit, en London

#### Para ser honestos, si modifico el parámetro "name" por "keyword" en los colegios, ganaría San Francisco, pero de esto me he dado cuenta demasiado tarde.

In [73]:
london

{'lat': 51.5214517, 'lng': -0.1420567}

## Recopilación:

### Vamos a meter todas las ubicaciones de los elementos que hemos comparado: colegios, bares, restaurantes veganos, etc en un dataframe, en este paso por supuesto no nos olvidaremos de buscar una buena peluqueria canina para Dobby y un campo de basket para Willy

### Modificación:
#### Inicialmente iba a mostrar los datos almacenados en "neneslond", "starslond", pero creo que quedará un mapa más completo si mostramos más elementos. London ya ha ganado en la comparativa, así que mostraré los primeros 20 resultados para cada búsqueda en Google Places


### Así que primero le preguntaré a google, y luego utilizaré la función y el bucle vistos en la clase de "geo-dataframe" para crear los dataframes con los datos de forma legible para la visualización.


In [ ]:
# el número de 20 para mostrar es porque es el tamaño de la respuesta de google, se puede amploar 
# hasta 60 mediante el parametro "pagetoken" en una segunda y tercera búsqueda, que nos muestra 20 resultados omitiendo los
# mostrados en las anteriores, una suerte de "pasar página"

#### Sencillamente es utilizar la misma consulta inicial modificando el radius por rank_by

In [74]:
party2lond= gmaps.places_nearby(
    
        location= london,
        keyword = ["party"],
        type = ["bar"],
        rank_by= "distance"
        #page_token = party2lond.next_page_token, si realizamos primero la query como está, y luego hacemos esta, nos daría 20
        # resultados extra
        )

In [75]:
vegans2lond= gmaps.places_nearby(
    
        location= london,
        keyword = ["vegan"],
        type = ["restaurant"],
        rank_by= "distance"
        )

In [76]:
stars2lond= gmaps.places_nearby(
    
        location = london,
        name = ["starbucks"],
        rank_by = "distance"
        )

In [77]:
padobby= gmaps.places_nearby(
        location = london,
        keyword=["pet groomet", "dog groomet", "pet beauty salon"],
        rank_by= "distance"
        )

In [78]:
pawilly= gmaps.places_nearby(
        location= london,
        name=["basketball stadium"],
        rank_by= "distance"
        )

#### Entramos primero en results, que es la lista en la que se guardan los resultados:

In [79]:
decodeparty2lond = party2lond["results"]
decodenenes2lond = neneslond2 ["results"] # recordamos que estos ya estaban hechos.
decodevegans2lond = vegans2lond ["results"]
decodestars2lond = stars2lond ["results"]
decodepadobby= padobby ["results"]
decodepawilly = pawilly ["results"]


In [80]:
decodepawilly 

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 51.5327517, 'lng': -0.1328685},
   'viewport': {'northeast': {'lat': 51.53403117989272,
     'lng': -0.1314531701072778},
    'southwest': {'lat': 51.53133152010727, 'lng': -0.1341528298927222}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
  'name': 'Somers Town Community Sports Centre',
  'opening_hours': {'open_now': False},
  'photos': [{'height': 3024,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113812005797241384690">Ash - R11 Rahman</a>'],
    'photo_reference': 'ATtYBwKy28dWUOWueDGc59MSfTd0Z1jkUx0xkmQQRaBpFuaGJe4Ij8-bCdPLcSpN4RcDwjOn4fyqqbyq7M-gTy7W3NiJmxfNlCI41uUIcKpSqDrYIblsY2gAUQ79pSCh1QV0UytmCMKH9rJ-s3GKJZQHVWKw_ySJv_soJkvwV6-dpieYZ3Ko',
    'width': 4032}],
  'place_id': 'ChIJpzmVryMbdkgRYEsmoTk8NfU',
  'plus_code': {'compound_code': 'GVM8+4V London, United Kingdom',
   'global_code': '9C3XGVM8+4V'},
  'rating': 4.4,
  're

#### Establecemos la ruta para cada campo que nos interesa y la almacenamos en variables:

In [81]:
mapa_nombre =  ["name"]
mapa_latitud = ["geometry", "location", "lat"]
mapa_longitud = ["geometry", "location", "lng"]
mapa_rating = [ "rating"]

#### Definimos la función que entrará en el camino indicado anteriormente

In [89]:
def getFromDict(diccionario,mapa):
    """" 
    Esta función recibe 2 parámetros, un diccionario de diccionarios y una lista de strings  realizada "ad hoc" para 
        ese diccionario, esos strings serán las keys por las que entrará la función para retornarnos el value del último string
    """
    return reduce(operator.getitem,mapa,diccionario)

#### Y ahora crearemos un diccionario, y con un bucle for que utiliza la función anterior lo rellenaremos: 
##### No lo he hecho función porque el key "Category" lo voy a rellenar a mano, la flexibilidad que nos ofrece google places a la hora de realizar querys, aquí se vuelve en contra, no existe un "type" o una "keyword" unico que nos pueda servir en esta situación

In [90]:
# no lo he hecho función porque el key "Category" lo voy a rellenar a mano
dicparty = []
for dic in decodeparty2lond:
    paralista = {}
    paralista["Name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    paralista["Rating"] = getFromDict (dic, mapa_rating)
    paralista["Category"]= "Bares_pubs"
    dicparty.append(paralista)

In [91]:
dfparty = pd.DataFrame(dicparty)
dfparty.head() 

,Name,latitud,longitud,Rating,Category
0,Bibbers,51.521683,-0.143679,0.0,Bares_pubs
1,Simmons Bar | Fitzrovia,51.522198,-0.138771,4.3,Bares_pubs
2,Nordic Bar,51.517858,-0.135694,4.0,Bares_pubs
3,Mimi's Bar & Lounge,51.517500,-0.135278,3.2,Bares_pubs
4,Simmons Bar | Oxford Street,51.515663,-0.136031,4.4,Bares_pubs


In [92]:
dicnenes = []
for dic in decodenenes2lond:
    paralista = {}
    paralista["Name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    paralista["Rating"] = getFromDict (dic, mapa_rating)
    paralista["Category"]= "Instituciones_educativas"
    dicnenes.append(paralista)

In [93]:
dicvegans = []
for dic in decodevegans2lond:
    paralista = {}
    paralista["Name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    paralista["Rating"] = getFromDict (dic, mapa_rating)
    paralista["Category"]= "Restaurantes_veganos"
    dicvegans.append(paralista)

In [94]:
dicstars = []
for dic in decodestars2lond:
    paralista = {}
    paralista["Name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    paralista["Rating"] = getFromDict (dic, mapa_rating)
    paralista["Category"]= "Starbucks"
    dicstars.append(paralista)

In [95]:
dicdobby = []
for dic in decodepadobby:
    paralista = {}
    paralista["Name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    paralista["Rating"] = getFromDict (dic, mapa_rating)
    paralista["Category"]= "Peluquerias de mascotas"
    dicdobby.append(paralista)

In [96]:
dicwilly = []
for dic in decodepawilly:
    paralista = {}
    paralista["Name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    paralista["Rating"] = getFromDict (dic, mapa_rating)
    paralista["Category"]= "Estadios de basket"
    dicwilly.append(paralista)

#### Ahora lo almacenaremos en DataFrames y los concatenaremos sin mayor problema

In [97]:
dfnenes = pd.DataFrame (dicnenes)
dfvegans = pd.DataFrame (dicvegans)
dfstars = pd.DataFrame (dicstars)
dfdobby = pd.DataFrame (dicdobby)
dfwilly = pd.DataFrame (dicwilly)

In [98]:
dfvisualizacion= pd.concat ([dfnenes, dfvegans,dfstars, dfdobby, dfwilly, dfparty], ignore_index=True)

In [99]:
dfvisualizacion.sample (10)

,Name,latitud,longitud,Rating,Category
92,GO Mammoth,51.517956,-0.229341,2.8,Estadios de basket
14,Rangers Kindergarten,51.526033,-0.053703,5.0,Instituciones_educativas
117,Mobile Bar Hire,51.501017,-0.129633,0.0,Bares_pubs
118,The Chapel Bar,51.532721,-0.111644,4.3,Bares_pubs
82,Hireapitch UK ltd.,51.500955,-0.108892,3.7,Estadios de basket
25,Govinda's Pure Vegetarian Restaurant,51.515835,-0.132760,4.3,Restaurantes_veganos
57,Starbucks Coffee,51.515233,-0.130191,3.9,Starbucks
105,The Roxy,51.516607,-0.133026,3.1,Bares_pubs
80,Somers Town Community Sports Centre,51.532752,-0.132869,4.4,Estadios de basket
22,Vantra Eden,51.521040,-0.135123,4.8,Restaurantes_veganos


## Visualización: 
#### En este apartado voy a intentar utilizar keplerg para visionar Londres, donde centraré el mapa en las coordenadas ganadoras y mostraré todos los elementos almacenados en el apartado anterior.




#### Primero crearé un mapa base, y luego cambiaré la configuración para incluir nuestro df

In [14]:
# he borrado los intentos de copiar el config... pero en resumen, no he podido

#### Una vez intentado lo anterior, vamos a configurarlo a mano, mediante la interfaz, por lo que no habrá codigo.

In [103]:
map_2= KeplerGl(height=700, data={'data_1': dfvisualizacion}, config= {'version': 'v1',
 'config': {'visState': {'filters': [{'dataId': ['data_1'],
     'id': 'xxwmgl5kh',
     'name': ['Category'],
     'type': 'multiSelect',
     'value': ['Restaurantes_veganos'],
     'enlarged': False,
     'plotType': 'histogram',
     'animationWindow': 'free',
     'yAxis': None}],
   'layers': [{'id': 'geocoder_layer',
     'type': 'icon',
     'config': {'dataId': 'geocoder_dataset',
      'label': 'Geocoder Layer',
      'color': [255, 0, 0],
      'columns': {'lat': 'lt', 'lng': 'ln', 'icon': 'icon'},
      'isVisible': True,
      'visConfig': {'radius': 80,
       'fixedRadius': False,
       'opacity': 0.8,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radiusRange': [0, 50]},
      'hidden': True,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}},
    {'id': 'xp2uxl4',
     'type': 'point',
     'config': {'dataId': 'data_1',
      'label': 'Nombre',
      'color': [201, 0, 0],
      'columns': {'lat': 'latitud', 'lng': 'longitud', 'altitude': None},
      'isVisible': True,
      'visConfig': {'radius': 10.8,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'thickness': 2,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radiusRange': [0, 50],
       'filled': True},
      'hidden': False,
      'textLabel': [{'field': {'name': 'Name', 'type': 'string'},
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}},
    {'id': 'kqopx8w',
     'type': 'point',
     'config': {'dataId': 'data_1',
      'label': 'categoria',
      'color': [47, 197, 204],
      'columns': {'lat': 'latitud', 'lng': 'longitud', 'altitude': None},
      'isVisible': False,
      'visConfig': {'radius': 10,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'thickness': 2,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radiusRange': [0, 50],
       'filled': True},
      'hidden': False,
      'textLabel': [{'field': {'name': 'Category', 'type': 'string'},
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'data_1': [],
      'geocoder_dataset': [{'name': 'lt', 'format': None},
       {'name': 'ln', 'format': None},
       {'name': 'icon', 'format': None},
       {'name': 'text', 'format': None}]},
     'compareMode': False,
     'compareType': 'relative',
     'enabled': False},
    'brush': {'size': 7.9, 'enabled': True},
    'geocoder': {'enabled': True},
    'coordinate': {'enabled': False}},
   'layerBlending': 'additive',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 0,
   'dragRotate': False,
   'latitude': 51.51847899999999,
   'longitude': -0.10982750000001082,
   'pitch': 0,
   'zoom': 12,
   'isSplit': False},
  'mapStyle': {'styleType': 'muted_night',
   'topLayerGroups': {'label': True},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [26.848523094644484,
    31.1442867897876,
    35.440050484930715],
   'mapStyles': {}}}})
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['data_1'], 'id': 'xxwmgl5kh'…

In [618]:
map_2.config # configuración inicial, o casi

{'version': 'v1',
 'config': {'visState': {'filters': [{'dataId': ['data_1'],
     'id': 'ibtlzx5f8',
     'name': ['Category'],
     'type': 'multiSelect',
     'value': ['Instituciones_educativas'],
     'enlarged': False,
     'plotType': 'histogram',
     'animationWindow': 'free',
     'yAxis': None}],
   'layers': [],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'data_1': [{'name': 'latitud',
        'format': None},
       {'name': 'longitud', 'format': None}]},
     'compareMode': True,
     'compareType': 'absolute',
     'enabled': False},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': True},
    'coordinate': {'enabled': True}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 0,
   'dragRotate': False,
   'latitude': 51.51125911319848,
   'longitude': -0.14327251417106815,
   'pitch': 0,
   'zoom': 12.819258982123904,
   'isSplit': False},
  'mapStyle': {

In [641]:
map_2.config #configuración final

{'version': 'v1',
 'config': {'visState': {'filters': [{'dataId': ['data_1'],
     'id': 'xxwmgl5kh',
     'name': ['Category'],
     'type': 'multiSelect',
     'value': ['Restaurantes_veganos'],
     'enlarged': False,
     'plotType': 'histogram',
     'animationWindow': 'free',
     'yAxis': None}],
   'layers': [{'id': 'geocoder_layer',
     'type': 'icon',
     'config': {'dataId': 'geocoder_dataset',
      'label': 'Geocoder Layer',
      'color': [255, 0, 0],
      'columns': {'lat': 'lt', 'lng': 'ln', 'icon': 'icon'},
      'isVisible': True,
      'visConfig': {'radius': 80,
       'fixedRadius': False,
       'opacity': 0.8,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radiusRange': [0, 50]},
      'hidden': True,
      'textLabel': [{'field': None,
        'color': [255, 255, 

In [102]:
map_2.save_to_html(file_name = "..\mapa\mapalond.html", read_only = False)

Map saved to ..\mapa\mapalond.html!


#### Para el resultado final me hubiera gustado poder colores distintos para cada categoria distinta, pero no lo he conseguido. 